In [ ]:
# change the following to %matplotlib notebook for interactive plotting
%matplotlib inline

import numpy as np
import pandas as pd

import pims
import trackpy as tp
import os

import matplotlib  as mpl 
import matplotlib.pyplot as plt 

import skimage
import skimage.measure
import matplotlib.patches as mpatches

from scipy import ndimage
from skimage import morphology, util, filters

import tifffile
from bokeh.models.tickers import FixedTicker

import numpy as np
from bokeh.plotting import show, figure
from bokeh.io import output_notebook
from bokeh.models import ColorBar, LinearColorMapper, ColumnDataSource, BasicTicker
from bokeh.transform import transform
import matplotlib as mpl
import matplotlib.pyplot as plt


# Optionally, tweak styles.
mpl.rc('figure',  figsize=(10, 6))
mpl.rc('image', cmap='gray')

### Load image stack

In [ ]:
# load tiff stake with pims. Notice here I only have two channels, gfp and cy5
#Burst /Volumes/SL_2023/0313/droplet_different_low_MT_6_8_2_burst

frames = pims.ImageSequence('/Users/scliu/Dropbox/Academics/PhD_phase/Thomson_Lab/local_to_global_pre-print/data/figure_4/aster_connection/5min_no_merge/*.tif')




In [ ]:
# only want the bf frame
bf_frames = frames[::2]
bf_frames[1]

In [ ]:
from skimage import filters, util
from scipy import ndimage
import numpy as np

def create_pipeline():
    frame_counter = {'count': 0}  # use a dict to make it mutable inside the nested function

    @pims.pipeline
    def preprocess_foam(img):
        """
        Apply image processing functions to return a binary image
        """
        # Apply thresholds
        percentile = 99.3 if frame_counter['count'] < 29 else 99.99999  # adjust the percentile based on the frame number
        threshold_value = np.percentile(img, percentile)  # calculate the percentile
        idx = img < threshold_value  # consider pixels brighter than the percentile
        idx2 = img >= threshold_value  # consider pixels not brighter than the percentile
        img[idx] = 255
        img[idx2] = 0
        img = ndimage.binary_dilation(img)
        img = ndimage.binary_dilation(img)

        # increment the frame counter
        frame_counter['count'] += 1

        return util.img_as_int(img)
    
    return preprocess_foam

preprocess_foam = create_pipeline()



frames_thresh = preprocess_foam(bf_frames)
plt.imshow(frames_thresh[11]);

In [ ]:
# First, create two separate pipelines for each percentile

def create_pipeline(percentile):
    @pims.pipeline
    def preprocess_foam(img):
        """
        Apply image processing functions to return a binary image
        """
        threshold_value = np.percentile(img, percentile)  # calculate the percentile
        idx = img < threshold_value  # consider pixels brighter than the percentile
        idx2 = img >= threshold_value  # consider pixels not brighter than the percentile
        img[idx] = 255
        img[idx2] = 0
        img = ndimage.binary_dilation(img)
        img = ndimage.binary_dilation(img)
        return util.img_as_int(img)
    return preprocess_foam

# Then, process the first 30 frames with 99 percentile and the remaining frames with 99.95 percentile

preprocess_foam_994 = create_pipeline(99.4)
preprocess_foam_995 = create_pipeline(99.5)
preprocess_foam_9997 = create_pipeline(99.97)

frames_thresh_994 = preprocess_foam_994(bf_frames[:20])
frames_thresh_995 = preprocess_foam_995(bf_frames[20:30])
frames_thresh_9997 = preprocess_foam_9997(bf_frames[30:])

frames_thresh = np.concatenate((frames_thresh_994, frames_thresh_995, frames_thresh_9997), axis=0)

# Now, frames_thresh should contain all frames processed with the appropriate percentile
plt.imshow(frames_thresh[5]);


In [ ]:
len(bf_frames)

In [ ]:
img_example = frames_thresh[18]

# Label elements on the picture
white = 255
label_image, number_of_labels = skimage.measure.label(img_example, background=white, return_num=True)
print("Found %d features"%(number_of_labels))
fig, ax = plt.subplots(ncols=1, nrows=1, figsize=(12, 12))
ax.imshow(img_example)
for region in skimage.measure.regionprops(label_image, intensity_image=img_example):
    # Everywhere, skip small and large areas
    if region.area < 300 or region.area > 8000:
        continue
    # Only black areas
    if region.mean_intensity > 1:
        continue
    # On the top, skip small area with a second threshold
    if region.centroid[0] > 8000 and region.area < 300:
        continue


    # Draw rectangle which survived to the criterions
    minr, minc, maxr, maxc = region.bbox
    rect = mpatches.Rectangle((minc, minr), maxc - minc, maxr - minr,
                              fill=False, edgecolor='red', linewidth=1)

    ax.add_patch(rect)

In [ ]:

def filter_regions(num, img):
    white = 255
    label_image = skimage.measure.label(img, background=white)
    regions = skimage.measure.regionprops(label_image, intensity_image=img)

    filtered_regions = [
        {
            'y': region.centroid[0],
            'x': region.centroid[1],
            'frame': num
        }
        for region in regions
        if (
            200 <= region.area <= 5000 and
            region.mean_intensity <= 1 and
            not (region.centroid[0] > 5000 and region.area < 200)
            )
    ]

    return filtered_regions


filtered_regions_list = [filter_regions(num, img) for num, img in enumerate(frames_thresh)]
features = pd.DataFrame([item for sublist in filtered_regions_list for item in sublist])


In [ ]:
def plot_features_with_labels(frame, features_df, image):
    """
    This function annotates each detected feature with its ID on the plot.

    Parameters:
    frame: The number of the frame you want to plot.
    features_df: A pandas DataFrame that contains the features detected by trackpy.
    image: The image you want to plot the features on.
    """
    # Create a new DataFrame that only contains the features in the frame you're interested in
    frame_features = features_df[features_df.frame==frame]
    
    fig, ax = plt.subplots(figsize=(12, 12))
    ax.imshow(image)
    tp.annotate(frame_features, image, ax=ax)
    
    # Annotate the ID of each feature
    for index, feature in frame_features.iterrows():
        ax.text(feature.x, feature.y, str(index), color='red')

    plt.show()


# Then you would use it like this:
id_example = 1
plot_features_with_labels(18, features, img_example)


In [ ]:
id_example = 50
tp.annotate(features[features.frame==(id_example+1)], img_example);


In [ ]:
features

In [ ]:
search_range = 40
t = tp.link_df(features, search_range, memory=40)
tp.plot_traj(t, superimpose=frames_thresh[50])

In [ ]:
def plot_trajectory_with_labels(particles_df):
    """
    This function plots the trajectories of particles and annotates each one with its ID.

    Parameters:
    particles_df: A pandas DataFrame that contains the particles identified by trackpy.
    """
    fig, ax = plt.subplots(figsize=(10, 10))
    tp.plot_traj(particles_df, ax=ax)

    # Get the last position of each particle to place the text
    last_positions = particles_df.groupby('particle').last()

    # Annotate the ID of each particle
    for particle, position in last_positions.iterrows():
        ax.text(position.x, position.y, str(int(particle)), color='red')

    plt.show()

# Then you would use it like this:
plot_trajectory_with_labels(t)


In [ ]:
particle_ids = [10, 11]  # Replace with the IDs of the particles you want to plot
selected = t[t['particle'].isin(particle_ids)]
# Filter rows where particle is 24 and index is between 50 and 60
# Print the filtered dataframe
tp.plot_traj(selected, superimpose=frames_thresh[70])


In [ ]:
import matplotlib.ticker as ticker

# Ensure data is sorted by frame
selected_sorted = selected.sort_values(by='frame')

# Initialize empty list for storing particle data
particle_data = []

# Loop through each particle ID
for particle_id in particle_ids:
    # Filter for current particle ID
    particle = selected_sorted[selected_sorted['particle'] == particle_id]
    
    # Store x, y, and time data for current particle. Convert pixels to um.
    particle_data.append((particle['x'].values * 0.43, particle['y'].values * 0.43, particle['frame'].values * 10/60))
    
# Convert list of tuples to numpy array
particle_data = np.array(particle_data)

# Create figure and axis
fig, ax = plt.subplots(figsize=(6, 5))  # Adjust as needed to get a square plot

# Loop through each particle and plot its trajectory
for i in range(len(particle_data)):
    sc = ax.scatter(particle_data[i, 0], particle_data[i, 1], c=particle_data[i, 2], cmap='jet')

# Invert y-axis and set plot limits
ax.set_xlim(0, 2048 * 0.43)
ax.set_ylim(0, 2048 * 0.43)
ax.tick_params(axis='both', labelsize=20)


# Set x and y ticks
ax.set_xticks([0, 2048 * 0.43 / 2, 2048 * 0.43])
ax.set_yticks([0, 2048 * 0.43 / 2, 2048 * 0.43])

formatter = ticker.FormatStrFormatter('%.0f')
# Apply the formatter to the x and y axes
ax.xaxis.set_major_formatter(formatter)
ax.yaxis.set_major_formatter(formatter)

# Set x and y labels
ax.set_xlabel('X (um)', fontsize=20)
ax.set_ylabel('Y (um)', fontsize=20)

# Add colorbar
cbar = plt.colorbar(sc, ax=ax)
cbar.set_label('Time (minutes)', fontsize=20)

plt.tight_layout()
plt.show()


In [ ]:
# Set a higher resolution for the figure
plt.figure(dpi=300)

# Define a clear font size for labels, ticks, and title
label_fontsize = 14
tick_fontsize = 12
title_fontsize = 16

# Use a different colormap if desired (e.g., "viridis" is perceptually uniform and colorblind-friendly)
cmap_choice = 'viridis'

# Loop through each particle and plot its trajectory
for particle in particle_data:
    x, y, frame_in_minutes = particle
    plt.scatter(x, y, c=frame_in_minutes, cmap=cmap_choice, vmin=np.min(t_filtered['frame'])*10/60, vmax=np.max(t_filtered['frame'])*10/60, edgecolor='k', linewidth=0.3)

# Add colorbar
cbar = plt.colorbar(label='Time (minutes)')
cbar.ax.tick_params(labelsize=tick_fontsize) 
cbar.set_label('Time (minutes)', size=label_fontsize)

# Invert y-axis and set plot limits
plt.gca().invert_yaxis()
plt.xlim(0, 2048 * 0.43)
plt.ylim(2048 * 0.43, 0)

# Set equal scaling for x and y axes
plt.gca().set_aspect('equal', adjustable='box')

# Setting tick parameters
plt.xticks(fontsize=tick_fontsize)
plt.yticks(fontsize=tick_fontsize)

# Set axis labels
plt.xlabel('X (pixel)', fontsize=label_fontsize)
plt.ylabel('Y (pixel)', fontsize=label_fontsize)

# Optionally, you can set a title
# plt.title('Particle Trajectories Over Time', fontsize=title_fontsize)

# Tight layout often improves the spacing between subplots
plt.tight_layout()

# Display the plot
plt.show()


In [ ]:


# Activate inline plotting in Jupyter
output_notebook()

# Extract RGB values from Matplotlib's 'jet' colormap
colormap = plt.get_cmap("jet")
bokeh_palette = [mpl.colors.rgb2hex(m) for m in colormap(np.arange(colormap.N))]

# Create a Bokeh figure with matching aspect ratio
# Increase the width to account for color bar
p = figure(plot_width=500, plot_height=500, tools="pan,reset,save,wheel_zoom")

# Define color mapper using the 'jet' colormap from Matplotlib
color_mapper = LinearColorMapper(palette=bokeh_palette, low=0, high=np.max(selected['frame']*10/60))

for particle in particle_data:
    x, y, frame = particle
    source = ColumnDataSource(data=dict(x=x, y=y, color=frame))
    p.scatter(x='x', y='y', source=source, size=8, color=transform('color', color_mapper))


color_bar = ColorBar(color_mapper=color_mapper, width=370, location=(0,0), 
                     title='Time (minutes)', ticker=FixedTicker(ticks=[0, np.max(selected['frame']*10/60)/2, np.max(selected['frame']*10/60)]))
color_bar.major_label_text_font_size = '16pt'  # Bigger color bar labels
p.add_layout(color_bar, 'above')
color_bar.title_text_font_size = '18pt'  # Set to desired size


# Adjust the view settings
p.y_range.flipped = True
p.x_range.start, p.x_range.end = 0, 2048 * 0.43
p.y_range.start, p.y_range.end = 0, 2048 * 0.43

# Set axis labels with bigger fonts
p.xaxis.axis_label = "X (µm)"
p.yaxis.axis_label = "Y (µm)"
p.xaxis.axis_label_text_font_size = '18pt'
p.yaxis.axis_label_text_font_size = '18pt'
p.xaxis.major_label_text_font_size = '16pt'
p.yaxis.major_label_text_font_size = '16pt'
# Set specific ticker values
p.xaxis.ticker = FixedTicker(ticks=[0, 440, 880])
p.yaxis.ticker = FixedTicker(ticks=[0, 440, 880])

show(p)



# Connecting Aster

In [ ]:
# load tiff stake with pims. Notice here I only have two channels, gfp and cy5
#Burst /Volumes/SL_2023/0313/droplet_different_low_MT_6_8_2_burst

frames = pims.ImageSequence('/Users/scliu/Dropbox/Academics/PhD_phase/Thomson_Lab/local_to_global_pre-print/data/figure_4/aster_connection/90min_merged/*.tif')
# only want the bf frame
bf_frames = frames[::2]
bf_frames[1]



In [ ]:
from skimage import filters, util
from scipy import ndimage
import numpy as np

def create_pipeline():
    frame_counter = {'count': 0}  # use a dict to make it mutable inside the nested function

    @pims.pipeline
    def preprocess_foam(img):
        """
        Apply image processing functions to return a binary image
        """
        # Apply thresholds
        percentile = 99.8 if frame_counter['count'] < 29 else 99.8  # adjust the percentile based on the frame number
        threshold_value = np.percentile(img, percentile)  # calculate the percentile
        idx = img < threshold_value  # consider pixels brighter than the percentile
        idx2 = img >= threshold_value  # consider pixels not brighter than the percentile
        img[idx] = 255
        img[idx2] = 0
        img = ndimage.binary_dilation(img)
        img = ndimage.binary_dilation(img)

        # increment the frame counter
        frame_counter['count'] += 1

        return util.img_as_int(img)
    
    return preprocess_foam

preprocess_foam = create_pipeline()



frames_thresh = preprocess_foam(bf_frames)
plt.imshow(frames_thresh[20]);

In [ ]:
# First, create two separate pipelines for each percentile

def create_pipeline(percentile):
    @pims.pipeline
    def preprocess_foam(img):
        """
        Apply image processing functions to return a binary image
        """
        threshold_value = np.percentile(img, percentile)  # calculate the percentile
        idx = img < threshold_value  # consider pixels brighter than the percentile
        idx2 = img >= threshold_value  # consider pixels not brighter than the percentile
        img[idx] = 255
        img[idx2] = 0
        img = ndimage.binary_dilation(img)
        img = ndimage.binary_dilation(img)
        return util.img_as_int(img)
    return preprocess_foam

# Then, process the first 30 frames with 99 percentile and the remaining frames with 99.95 percentile

preprocess_foam_994 = create_pipeline(99.4)
preprocess_foam_995 = create_pipeline(99.5)
preprocess_foam_9997 = create_pipeline(99.97)

frames_thresh_994 = preprocess_foam_994(bf_frames[:20])
frames_thresh_995 = preprocess_foam_995(bf_frames[20:30])
frames_thresh_9997 = preprocess_foam_9997(bf_frames[30:])

frames_thresh = np.concatenate((frames_thresh_994, frames_thresh_995, frames_thresh_9997), axis=0)

# Now, frames_thresh should contain all frames processed with the appropriate percentile
plt.imshow(frames_thresh[50]);


In [ ]:
plt.imshow(frames_thresh[80]);

In [ ]:
img_example = frames_thresh[18]

# Label elements on the picture
white = 255
label_image, number_of_labels = skimage.measure.label(img_example, background=white, return_num=True)
print("Found %d features"%(number_of_labels))
fig, ax = plt.subplots(ncols=1, nrows=1, figsize=(12, 12))
ax.imshow(img_example)
for region in skimage.measure.regionprops(label_image, intensity_image=img_example):
    # Everywhere, skip small and large areas
    if region.area < 300 or region.area > 8000:
        continue
    # Only black areas
    if region.mean_intensity > 1:
        continue
    # On the top, skip small area with a second threshold
    if region.centroid[0] > 8000 and region.area < 300:
        continue


    # Draw rectangle which survived to the criterions
    minr, minc, maxr, maxc = region.bbox
    rect = mpatches.Rectangle((minc, minr), maxc - minc, maxr - minr,
                              fill=False, edgecolor='red', linewidth=1)

    ax.add_patch(rect)

In [ ]:

def filter_regions(num, img):
    white = 255
    label_image = skimage.measure.label(img, background=white)
    regions = skimage.measure.regionprops(label_image, intensity_image=img)

    filtered_regions = [
        {
            'y': region.centroid[0],
            'x': region.centroid[1],
            'frame': num
        }
        for region in regions
        if (
            200 <= region.area <= 5000 and
            region.mean_intensity <= 1 and
            not (region.centroid[0] > 5000 and region.area < 200)
            )
    ]

    return filtered_regions


filtered_regions_list = [filter_regions(num, img) for num, img in enumerate(frames_thresh)]
features = pd.DataFrame([item for sublist in filtered_regions_list for item in sublist])


In [ ]:
def plot_features_with_labels(frame, features_df, image):
    """
    This function annotates each detected feature with its ID on the plot.

    Parameters:
    frame: The number of the frame you want to plot.
    features_df: A pandas DataFrame that contains the features detected by trackpy.
    image: The image you want to plot the features on.
    """
    # Create a new DataFrame that only contains the features in the frame you're interested in
    frame_features = features_df[features_df.frame==frame]
    
    fig, ax = plt.subplots(figsize=(12, 12))
    ax.imshow(image)
    tp.annotate(frame_features, image, ax=ax)
    
    # Annotate the ID of each feature
    for index, feature in frame_features.iterrows():
        ax.text(feature.x, feature.y, str(index), color='red')

    plt.show()


# Then you would use it like this:
id_example = 1
plot_features_with_labels(18, features, img_example)


In [ ]:
id_example = 50
tp.annotate(features[features.frame==(id_example+1)], img_example);


In [ ]:
search_range = 40
t = tp.link_df(features, search_range, memory=40)
tp.plot_traj(t, superimpose=frames_thresh[50])

In [ ]:
def plot_trajectory_with_labels(particles_df):
    """
    This function plots the trajectories of particles and annotates each one with its ID.

    Parameters:
    particles_df: A pandas DataFrame that contains the particles identified by trackpy.
    """
    fig, ax = plt.subplots(figsize=(10, 10))
    tp.plot_traj(particles_df, ax=ax)

    # Get the last position of each particle to place the text
    last_positions = particles_df.groupby('particle').last()

    # Annotate the ID of each particle
    for particle, position in last_positions.iterrows():
        ax.text(position.x, position.y, str(int(particle)), color='red')

    plt.show()

# Then you would use it like this:
plot_trajectory_with_labels(t)


In [ ]:
particle_ids = [5, 11]  # Replace with the IDs of the particles you want to plot
selected = t[t['particle'].isin(particle_ids)]
# Filter rows where particle is 24 and index is between 50 and 60
# Print the filtered dataframe
tp.plot_traj(selected, superimpose=frames_thresh[70])


In [ ]:
import matplotlib.ticker as ticker

# Ensure data is sorted by frame
selected_sorted = selected.sort_values(by='frame')

# Initialize empty list for storing particle data
particle_data = []

# Loop through each particle ID
for particle_id in particle_ids:
    # Filter for current particle ID
    particle = selected_sorted[selected_sorted['particle'] == particle_id]
    
    # Store x, y, and time data for current particle. Convert pixels to um.
    particle_data.append((particle['x'].values * 0.43, particle['y'].values * 0.43, particle['frame'].values * 10/60))
    
# Convert list of tuples to numpy array
particle_data = np.array(particle_data)

# Create figure and axis
fig, ax = plt.subplots(figsize=(6, 5))  # Adjust as needed to get a square plot

# Loop through each particle and plot its trajectory
for i in range(len(particle_data)):
    sc = ax.scatter(particle_data[i, 0], particle_data[i, 1], c=particle_data[i, 2], cmap='jet')

# Invert y-axis and set plot limits
ax.set_xlim(0, 2048 * 0.43)
ax.set_ylim(0, 2048 * 0.43)
ax.tick_params(axis='both', labelsize=20)


# Set x and y ticks
ax.set_xticks([0, 2048 * 0.43 / 2, 2048 * 0.43])
ax.set_yticks([0, 2048 * 0.43 / 2, 2048 * 0.43])

formatter = ticker.FormatStrFormatter('%.0f')
# Apply the formatter to the x and y axes
ax.xaxis.set_major_formatter(formatter)
ax.yaxis.set_major_formatter(formatter)

# Set x and y labels
ax.set_xlabel('X (um)', fontsize=20)
ax.set_ylabel('Y (um)', fontsize=20)

# Add colorbar
cbar = plt.colorbar(sc, ax=ax)
cbar.set_label('Time (minutes)', fontsize=20)

plt.tight_layout()
plt.show()


In [ ]:


# Activate inline plotting in Jupyter
output_notebook()

# Extract RGB values from Matplotlib's 'jet' colormap
colormap = plt.get_cmap("jet")
bokeh_palette = [mpl.colors.rgb2hex(m) for m in colormap(np.arange(colormap.N))]

# Create a Bokeh figure with matching aspect ratio
# Increase the width to account for color bar
p = figure(plot_width=500, plot_height=500, tools="pan,reset,save,wheel_zoom")

# Define color mapper using the 'jet' colormap from Matplotlib
color_mapper = LinearColorMapper(palette=bokeh_palette, low=0, high=np.max(selected['frame']*10/60))

for particle in particle_data:
    x, y, frame = particle
    source = ColumnDataSource(data=dict(x=x, y=y, color=frame))
    p.scatter(x='x', y='y', source=source, size=8, color=transform('color', color_mapper))


color_bar = ColorBar(color_mapper=color_mapper, width=370, location=(0,0), 
                     title='Time (minutes)', ticker=FixedTicker(ticks=[0, np.max(selected['frame']*10/60)/2, np.max(selected['frame']*10/60)]))
color_bar.major_label_text_font_size = '16pt'  # Bigger color bar labels
p.add_layout(color_bar, 'above')
color_bar.title_text_font_size = '18pt'  # Set to desired size


# Adjust the view settings
p.y_range.flipped = True
p.x_range.start, p.x_range.end = 0, 2048 * 0.43
p.y_range.start, p.y_range.end = 0, 2048 * 0.43

# Set axis labels with bigger fonts
p.xaxis.axis_label = "X (µm)"
p.yaxis.axis_label = "Y (µm)"
p.xaxis.axis_label_text_font_size = '18pt'
p.yaxis.axis_label_text_font_size = '18pt'
p.xaxis.major_label_text_font_size = '16pt'
p.yaxis.major_label_text_font_size = '16pt'
# Set specific ticker values
p.xaxis.ticker = FixedTicker(ticks=[0, 440, 880])
p.yaxis.ticker = FixedTicker(ticks=[0, 440, 880])

show(p)



# Moving aster

In [ ]:
frames = pims.ImageSequence('/Users/scliu/Dropbox/Academics/PhD_phase/Thomson_Lab/local_to_global_pre-print/data/figure_4/aster_move/global/*.tif')
# only want the bf frame
bf_frames = frames[::2]
bf_frames[1]

In [ ]:
from skimage import filters, util
from scipy import ndimage
import numpy as np

def create_pipeline():
    frame_counter = {'count': 0}  # use a dict to make it mutable inside the nested function

    @pims.pipeline
    def preprocess_foam(img):
        """
        Apply image processing functions to return a binary image
        """
        # Apply thresholds
        percentile = 99.8 if frame_counter['count'] < 29 else 99.8  # adjust the percentile based on the frame number
        threshold_value = np.percentile(img, percentile)  # calculate the percentile
        idx = img < threshold_value  # consider pixels brighter than the percentile
        idx2 = img >= threshold_value  # consider pixels not brighter than the percentile
        img[idx] = 255
        img[idx2] = 0
        img = ndimage.binary_dilation(img)
        img = ndimage.binary_dilation(img)

        # increment the frame counter
        frame_counter['count'] += 1

        return util.img_as_int(img)
    
    return preprocess_foam

preprocess_foam = create_pipeline()



frames_thresh = preprocess_foam(bf_frames)
plt.imshow(frames_thresh[20]);

In [ ]:
import ipywidgets as widgets
from IPython.display import display

def display_frame(i):
    plt.imshow(frames_thresh[i])
    plt.show()

# Create a slider to select frame number
slider = widgets.IntSlider(value=0, min=0, max=len(frames_thresh) - 1, step=1, description='Frame:')

# Interactively display the frame based on the slider
widgets.interactive(display_frame, i=slider)

In [ ]:
# First, create two separate pipelines for each percentile

def create_pipeline(percentile):
    @pims.pipeline
    def preprocess_foam(img):
        """
        Apply image processing functions to return a binary image
        """
        threshold_value = np.percentile(img, percentile)  # calculate the percentile
        idx = img < threshold_value  # consider pixels brighter than the percentile
        idx2 = img >= threshold_value  # consider pixels not brighter than the percentile
        img[idx] = 255
        img[idx2] = 0
        img = ndimage.binary_dilation(img)
        img = ndimage.binary_dilation(img)
        return util.img_as_int(img)
    return preprocess_foam

# Then, process the first 30 frames with 99 percentile and the remaining frames with 99.95 percentile

preprocess_foam= create_pipeline(99.8)


frames_thresh= preprocess_foam_994(bf_frames)

# Now, frames_thresh should contain all frames processed with the appropriate percentile
plt.imshow(frames_thresh[50]);


In [ ]:
plt.imshow(frames_thresh[90]);


In [ ]:
img_example = frames_thresh[70]

# Label elements on the picture
white = 255
label_image, number_of_labels = skimage.measure.label(img_example, background=white, return_num=True)
print("Found %d features"%(number_of_labels))
fig, ax = plt.subplots(ncols=1, nrows=1, figsize=(12, 12))
ax.imshow(img_example)
for region in skimage.measure.regionprops(label_image, intensity_image=img_example):
    # Everywhere, skip small and large areas|
    if region.area < 1000 or region.area > 15000:
        continue
    # Only black areas
    if region.mean_intensity > 1:
        continue
    # On the top, skip small area with a second threshold
    if region.centroid[0] > 15000 and region.area < 1000:
        continue


    # Draw rectangle which survived to the criterions
    minr, minc, maxr, maxc = region.bbox
    rect = mpatches.Rectangle((minc, minr), maxc - minc, maxr - minr,
                              fill=False, edgecolor='red', linewidth=1)

    ax.add_patch(rect)

In [ ]:

def filter_regions(num, img):
    white = 255
    label_image = skimage.measure.label(img, background=white)
    regions = skimage.measure.regionprops(label_image, intensity_image=img)

    filtered_regions = [
        {
            'y': region.centroid[0],
            'x': region.centroid[1],
            'frame': num
        }
        for region in regions
        if (
            1000 <= region.area <= 15000 and
            region.mean_intensity <= 1 and
            not (region.centroid[0] > 15000 and region.area < 1000)
            )
    ]

    return filtered_regions


filtered_regions_list = [filter_regions(num, img) for num, img in enumerate(frames_thresh)]
features = pd.DataFrame([item for sublist in filtered_regions_list for item in sublist])


In [ ]:
def plot_features_with_labels(frame, features_df, image):
    """
    This function annotates each detected feature with its ID on the plot.

    Parameters:
    frame: The number of the frame you want to plot.
    features_df: A pandas DataFrame that contains the features detected by trackpy.
    image: The image you want to plot the features on.
    """
    # Create a new DataFrame that only contains the features in the frame you're interested in
    frame_features = features_df[features_df.frame==frame]
    
    fig, ax = plt.subplots(figsize=(12, 12))
    ax.imshow(image)
    tp.annotate(frame_features, image, ax=ax)
    
    # Annotate the ID of each feature
    for index, feature in frame_features.iterrows():
        ax.text(feature.x, feature.y, str(index), color='red')

    plt.show()


# Then you would use it like this:
id_example = 1
plot_features_with_labels(18, features, img_example)


In [ ]:
id_example = 50
tp.annotate(features[features.frame==(id_example+1)], img_example);


In [ ]:
search_range = 40
t = tp.link_df(features, search_range, memory=100)
tp.plot_traj(t, superimpose=frames_thresh[50])

In [ ]:
def plot_trajectory_with_labels(particles_df):
    """
    This function plots the trajectories of particles and annotates each one with its ID.

    Parameters:
    particles_df: A pandas DataFrame that contains the particles identified by trackpy.
    """
    fig, ax = plt.subplots(figsize=(10, 10))
    tp.plot_traj(particles_df, ax=ax)

    # Get the last position of each particle to place the text
    last_positions = particles_df.groupby('particle').last()

    # Annotate the ID of each particle
    for particle, position in last_positions.iterrows():
        ax.text(position.x, position.y, str(int(particle)), color='red')

    plt.show()

# Then you would use it like this:
plot_trajectory_with_labels(t)


In [ ]:
particle_ids = [3, 7, 2, 6, 8, 5]  # Replace with the IDs of the particles you want to plot
selected = t[t['particle'].isin(particle_ids)]
# Filter rows where particle is 24 and index is between 50 and 60
# Print the filtered dataframe
tp.plot_traj(selected, superimpose=frames_thresh[70])


In [ ]:
selected

In [ ]:
# Ensure data is sorted by frame
selected_sorted = selected.sort_values(by='frame')

# Initialize empty list for storing particle data
particle_data = []

# Loop through each particle ID
for particle_id in particle_ids:
    # Filter for current particle ID
    particle = selected_sorted[selected_sorted['particle'] == particle_id]
    
    # Store x, y, and time data for current particle. Convert pixels to um.
    particle_data.append((particle['x'].values * 0.43, particle['y'].values * 0.43, particle['frame'].values * 10/60))
    
# Convert list of tuples to numpy array
particle_data = np.array(particle_data)

# Create figure and axis
fig, ax = plt.subplots(figsize=(6, 5))  # Adjust as needed to get a square plot

# Loop through each particle and plot its trajectory
for i in range(len(particle_data)):
    sc = ax.scatter(particle_data[i, 0], particle_data[i, 1], c=particle_data[i, 2], cmap='jet')

# Invert y-axis and set plot limits
ax.set_xlim(0, 2048 * 0.43)
ax.set_ylim(0, 2048 * 0.43)
ax.tick_params(axis='both', labelsize=20)


# Set x and y ticks
ax.set_xticks([0, 2048 * 0.43 / 2, 2048 * 0.43])
ax.set_yticks([0, 2048 * 0.43 / 2, 2048 * 0.43])

formatter = ticker.FormatStrFormatter('%.0f')
# Apply the formatter to the x and y axes
ax.xaxis.set_major_formatter(formatter)
ax.yaxis.set_major_formatter(formatter)

# Set x and y labels
ax.set_xlabel('X (um)', fontsize=20)
ax.set_ylabel('Y (um)', fontsize=20)

# Add colorbar
cbar = plt.colorbar(sc, ax=ax)
cbar.set_label('Time (minutes)', fontsize=20)

plt.tight_layout()
plt.show()


In [ ]:
# Get unique particle IDs

# Initialize empty list for storing particle data
particle_data = []

# Loop through each particle ID
for particle_id in particle_ids:
    # Filter for current particle ID
    particle = t[t['particle'] == particle_id]
    
    # Store x, y, and frame data for current particle
    particle_data.append((particle['x'].values, particle['y'].values, particle['frame'].values))

# Check the particle_data
for i, data in enumerate(particle_data):
    print(f"Particle ID: {particle_ids[i]}, x: {data[0]}, y: {data[1]}, frame: {data[2]}")


In [ ]:
import numpy as np

# Filter dataframe for first 165 frames
t_filtered = t[t['frame'] < 165]

# Get unique particle IDs

# Initialize empty list for storing particle data
particle_data = []

# Loop through each particle ID
for particle_id in particle_ids:
    # Filter for current particle ID
    particle = t_filtered[t_filtered['particle'] == particle_id]
    
    # Store x, y, and frame data for current particle
    particle_data.append((particle['x'].values, particle['y'].values, particle['frame'].values*10/60))

# Loop through each particle and plot its trajectory
for particle in particle_data:
    x, y, frame = particle
    
    # Plot with colormap normalization
    plt.scatter(x, y, c=frame, cmap='jet', vmin=np.min(t_filtered['frame']*10/60), vmax=np.max(t_filtered['frame']*10/60))

plt.colorbar(label='Time (minutes)')

# Invert y-axis and set plot limits
plt.gca().invert_yaxis()
plt.xlim(0, 2048)
plt.ylim(2048, 0)

# Set equal scaling for x and y axes
plt.gca().set_aspect('equal', adjustable='box')

plt.show()


In [ ]:


# Activate inline plotting in Jupyter
output_notebook()

# Extract RGB values from Matplotlib's 'jet' colormap
colormap = plt.get_cmap("jet")
bokeh_palette = [mpl.colors.rgb2hex(m) for m in colormap(np.arange(colormap.N))]

# Create a Bokeh figure with matching aspect ratio
# Increase the width to account for color bar
p = figure(plot_width=500, plot_height=500, tools="pan,reset,save,wheel_zoom")

# Define color mapper using the 'jet' colormap from Matplotlib
color_mapper = LinearColorMapper(palette=bokeh_palette, low=0, high=np.max(t_filtered['frame']*10/60))

for particle in particle_data:
    x, y, frame = particle
    source = ColumnDataSource(data=dict(x=x*0.43, y=y*0.43, color=frame))
    p.scatter(x='x', y='y', source=source, size=8, color=transform('color', color_mapper))


color_bar = ColorBar(color_mapper=color_mapper, width=370, location=(0,0), 
                     title='Time (minutes)', ticker=FixedTicker(ticks=[0, np.max(t_filtered['frame']*10/60)/2, np.max(t_filtered['frame']*10/60)]))
color_bar.major_label_text_font_size = '16pt'  # Bigger color bar labels
p.add_layout(color_bar, 'above')
color_bar.title_text_font_size = '18pt'  # Set to desired size


# Adjust the view settings
p.y_range.flipped = True
p.x_range.start, p.x_range.end = 0, 2048 * 0.43
p.y_range.start, p.y_range.end = 0, 2048 * 0.43

# Set axis labels with bigger fonts
p.xaxis.axis_label = "X (µm)"
p.yaxis.axis_label = "Y (µm)"
p.xaxis.axis_label_text_font_size = '18pt'
p.yaxis.axis_label_text_font_size = '18pt'
p.xaxis.major_label_text_font_size = '16pt'
p.yaxis.major_label_text_font_size = '16pt'
# Set specific ticker values
p.xaxis.ticker = FixedTicker(ticks=[0, 440, 880])
p.yaxis.ticker = FixedTicker(ticks=[0, 440, 880])

show(p)



# Local moving

In [ ]:
frames = pims.ImageSequence('/Users/scliu/Dropbox/Academics/PhD_phase/Thomson_Lab/local_to_global_pre-print/data/figure_4/aster_move/local/*.tif')
# only want the bf frame
bf_frames = frames[::2]
bf_frames[1]

In [ ]:
def create_pipeline():
    frame_counter = {'count': 0}  # use a dict to make it mutable inside the nested function

    @pims.pipeline
    def preprocess_foam(img):
        """
        Apply image processing functions to return a binary image
        """
        # Apply thresholds
        percentile = 99.8 if frame_counter['count'] < 29 else 99.8  # adjust the percentile based on the frame number
        threshold_value = np.percentile(img, percentile)  # calculate the percentile
        idx = img < threshold_value  # consider pixels brighter than the percentile
        idx2 = img >= threshold_value  # consider pixels not brighter than the percentile
        img[idx] = 255
        img[idx2] = 0
        img = ndimage.binary_dilation(img)
        img = ndimage.binary_dilation(img)

        # increment the frame counter
        frame_counter['count'] += 1

        return util.img_as_int(img)
    
    return preprocess_foam

preprocess_foam = create_pipeline()



frames_thresh = preprocess_foam(bf_frames)
plt.imshow(frames_thresh[20]);

In [ ]:
def display_frame(i):
    plt.imshow(frames_thresh[i])
    plt.show()

# Create a slider to select frame number
slider = widgets.IntSlider(value=0, min=0, max=len(frames_thresh) - 1, step=1, description='Frame:')

# Interactively display the frame based on the slider
widgets.interactive(display_frame, i=slider)

In [ ]:
# First, create two separate pipelines for each percentile

def create_pipeline(percentile):
    @pims.pipeline
    def preprocess_foam(img):
        """
        Apply image processing functions to return a binary image
        """
        threshold_value = np.percentile(img, percentile)  # calculate the percentile
        idx = img < threshold_value  # consider pixels brighter than the percentile
        idx2 = img >= threshold_value  # consider pixels not brighter than the percentile
        img[idx] = 255
        img[idx2] = 0
        img = ndimage.binary_dilation(img)
        img = ndimage.binary_dilation(img)
        return util.img_as_int(img)
    return preprocess_foam

# Then, process the first 30 frames with 99 percentile and the remaining frames with 99.95 percentile

preprocess_foam= create_pipeline(99.8)


frames_thresh= preprocess_foam_994(bf_frames)

# Now, frames_thresh should contain all frames processed with the appropriate percentile
plt.imshow(frames_thresh[50]);


In [ ]:
img_example = frames_thresh[70]

# Label elements on the picture
white = 255
label_image, number_of_labels = skimage.measure.label(img_example, background=white, return_num=True)
print("Found %d features"%(number_of_labels))
fig, ax = plt.subplots(ncols=1, nrows=1, figsize=(12, 12))
ax.imshow(img_example)
for region in skimage.measure.regionprops(label_image, intensity_image=img_example):
    # Everywhere, skip small and large areas|
    if region.area < 500 or region.area > 8000:
        continue
    # Only black areas
    if region.mean_intensity > 1:
        continue
    # On the top, skip small area with a second threshold
    if region.centroid[0] > 8000 and region.area < 500:
        continue


    # Draw rectangle which survived to the criterions
    minr, minc, maxr, maxc = region.bbox
    rect = mpatches.Rectangle((minc, minr), maxc - minc, maxr - minr,
                              fill=False, edgecolor='red', linewidth=1)

    ax.add_patch(rect)

In [ ]:

def filter_regions(num, img):
    white = 255
    label_image = skimage.measure.label(img, background=white)
    regions = skimage.measure.regionprops(label_image, intensity_image=img)

    filtered_regions = [
        {
            'y': region.centroid[0],
            'x': region.centroid[1],
            'frame': num
        }
        for region in regions
        if (
            1000 <= region.area <= 10000 and
            region.mean_intensity <= 1 and
            not (region.centroid[0] > 1000 and region.area < 1000)
            )
    ]

    return filtered_regions


filtered_regions_list = [filter_regions(num, img) for num, img in enumerate(frames_thresh)]
features = pd.DataFrame([item for sublist in filtered_regions_list for item in sublist])


In [ ]:
def plot_features_with_labels(frame, features_df, image):
    """
    This function annotates each detected feature with its ID on the plot.

    Parameters:
    frame: The number of the frame you want to plot.
    features_df: A pandas DataFrame that contains the features detected by trackpy.
    image: The image you want to plot the features on.
    """
    # Create a new DataFrame that only contains the features in the frame you're interested in
    frame_features = features_df[features_df.frame==frame]
    
    fig, ax = plt.subplots(figsize=(12, 12))
    ax.imshow(image)
    tp.annotate(frame_features, image, ax=ax)
    
    # Annotate the ID of each feature
    for index, feature in frame_features.iterrows():
        ax.text(feature.x, feature.y, str(index), color='red')

    plt.show()


# Then you would use it like this:
id_example = 1
plot_features_with_labels(18, features, img_example)


In [ ]:
id_example = 50
tp.annotate(features[features.frame==(id_example+1)], img_example);


In [ ]:
search_range = 40
t = tp.link_df(features, search_range, memory=100)
tp.plot_traj(t, superimpose=frames_thresh[50])

In [ ]:
def plot_trajectory_with_labels(particles_df):
    """
    This function plots the trajectories of particles and annotates each one with its ID.

    Parameters:
    particles_df: A pandas DataFrame that contains the particles identified by trackpy.
    """
    fig, ax = plt.subplots(figsize=(10, 10))
    tp.plot_traj(particles_df, ax=ax)

    # Get the last position of each particle to place the text
    last_positions = particles_df.groupby('particle').last()

    # Annotate the ID of each particle
    for particle, position in last_positions.iterrows():
        ax.text(position.x, position.y, str(int(particle)), color='red')

    plt.show()

# Then you would use it like this:
plot_trajectory_with_labels(t)


In [ ]:
particle_ids = t_filtered['particle'].unique()
selected = t[t['particle'].isin(particle_ids)]


In [ ]:
tp.plot_traj(selected, superimpose=frames_thresh[50])

In [ ]:
# Ensure data is sorted by frame
selected_sorted = selected.sort_values(by='frame')

# Initialize empty list for storing particle data
particle_data = []

# Loop through each particle ID

# Loop through each particle ID
for particle_id in particle_ids:
    # Filter for current particle ID
    particle = selected_sorted[selected_sorted['particle'] == particle_id]
    
    # Adjust the time to reflect the "alive time" of each particle
    alive_time = (particle['frame'].values - particle['frame'].values[0]) * 10/60
    
    # Store x, y, and time data for current particle. Convert pixels to um.
    particle_data.append((particle['x'].values * 0.43, particle['y'].values * 0.43, alive_time))
    
 
# Convert list of tuples to numpy array
particle_data = np.array(particle_data)

# Create figure and axis
fig, ax = plt.subplots(figsize=(6, 5))  # Adjust as needed to get a square plot

# Loop through each particle and plot its trajectory
for i in range(len(particle_data)):
    sc = ax.scatter(particle_data[i, 0], particle_data[i, 1], c=particle_data[i, 2], cmap='jet')

# Invert y-axis and set plot limits
ax.set_xlim(0, 2048 * 0.43)
ax.set_ylim(0, 2048 * 0.43)
ax.tick_params(axis='both', labelsize=20)


# Set x and y ticks
ax.set_xticks([0, 2048 * 0.43 / 2, 2048 * 0.43])
ax.set_yticks([0, 2048 * 0.43 / 2, 2048 * 0.43])

formatter = ticker.FormatStrFormatter('%.0f')
# Apply the formatter to the x and y axes
ax.xaxis.set_major_formatter(formatter)
ax.yaxis.set_major_formatter(formatter)

# Set x and y labels
ax.set_xlabel('X (um)', fontsize=20)
ax.set_ylabel('Y (um)', fontsize=20)

# Add colorbar
cbar = plt.colorbar(sc, ax=ax)
cbar.set_label('Time (minutes)', fontsize=20)

plt.tight_layout()
plt.show()


In [ ]:
particle_data

In [ ]:


# Activate inline plotting in Jupyter
output_notebook()

# Extract RGB values from Matplotlib's 'jet' colormap
colormap = plt.get_cmap("jet")
bokeh_palette = [mpl.colors.rgb2hex(m) for m in colormap(np.arange(colormap.N))]

# Create a Bokeh figure with matching aspect ratio
# Increase the width to account for color bar
p = figure(plot_width=500, plot_height=500, tools="pan,reset,save,wheel_zoom")

# Define color mapper using the 'jet' colormap from Matplotlib
color_mapper = LinearColorMapper(palette=bokeh_palette, low=0, high=np.max(t_filtered['frame']*10/60))

for particle in particle_data:
    x, y, frame = particle
    source = ColumnDataSource(data=dict(x=x, y=y, color=frame))
    p.scatter(x='x', y='y', source=source, size=8, color=transform('color', color_mapper))


color_bar = ColorBar(color_mapper=color_mapper, width=370, location=(0,0), 
                     title='Time (minutes)', ticker=FixedTicker(ticks=[0, 13.5, 27]))
color_bar.major_label_text_font_size = '16pt'  # Bigger color bar labels
p.add_layout(color_bar, 'above')
color_bar.title_text_font_size = '18pt'  # Set to desired size


# Adjust the view settings
p.y_range.flipped = True
p.x_range.start, p.x_range.end = 0, 2048 * 0.43
p.y_range.start, p.y_range.end = 0, 2048 * 0.43

# Set axis labels with bigger fonts
p.xaxis.axis_label = "X (µm)"
p.yaxis.axis_label = "Y (µm)"
p.xaxis.axis_label_text_font_size = '18pt'
p.yaxis.axis_label_text_font_size = '18pt'
p.xaxis.major_label_text_font_size = '16pt'
p.yaxis.major_label_text_font_size = '16pt'
# Set specific ticker values
p.xaxis.ticker = FixedTicker(ticks=[0, 440, 880])
p.yaxis.ticker = FixedTicker(ticks=[0, 440, 880])

show(p)



In [ ]:
# Ensure data is sorted by frame
selected_sorted = selected.sort_values(by='frame')

# Initialize empty list for storing particle data
particle_data = []

# Loop through each particle ID

# Loop through each particle ID
for particle_id in particle_ids:
    # Filter for current particle ID
    particle = selected_sorted[selected_sorted['particle'] == particle_id]
    
    # Adjust the time to reflect the "alive time" of each particle
    alive_time = (particle['frame'].values - particle['frame'].values[0]) * 10/60
    print(alive_time)
    # Store x, y, and time data for current particle. Convert pixels to um.
    particle_data.append((particle['x'].values * 0.43, particle['y'].values * 0.43, alive_time))


In [ ]:
selected_sorted = selected.sort_values(by='frame')
selected_sorted[selected_sorted['particle'] == 1]

In [ ]:
from bokeh.plotting import figure, show, output_notebook
from bokeh.models import ColumnDataSource, HoverTool

# Activate inline plotting in Jupyter
output_notebook()

# Create a data source for Bokeh from the DataFrame
source = ColumnDataSource(data=dict(
    x=selected_sorted['frame'],
    y=selected_sorted['particle'],
    particle_id=selected_sorted['particle']
))

# Create a Bokeh figure
p = figure(plot_width=600, plot_height=400, title="Frames Each Particle Existed",
           x_axis_label='Frame', y_axis_label='Particle ID')

# Add a scatter plot to the figure
p.circle(x='x', y='y', source=source, size=6, alpha=0.6)

# Add a hover tool to display particle ID and frame when hovering over a point
hover = HoverTool()
hover.tooltips = [("Particle ID", "@particle_id"), ("Frame", "@x")]
p.add_tools(hover)

show(p)


In [ ]:
alive_time

In [ ]:
from bokeh.models import ColorBar, LinearColorMapper, FixedTicker, ColumnDataSource
from bokeh.plotting import figure, output_notebook, show
from bokeh.transform import transform

# Activate inline plotting in Jupyter
output_notebook()

# Extract RGB values from Matplotlib's 'jet' colormap
colormap = plt.get_cmap("jet")
bokeh_palette = [mpl.colors.rgb2hex(m) for m in colormap(np.arange(colormap.N))]

# Create a Bokeh figure
p = figure(plot_width=500, plot_height=500, tools="pan,reset,save,wheel_zoom")

# Define color mapper using the 'jet' colormap from Matplotlib
max_alive_time = np.max([np.max((selected_sorted[selected_sorted['particle'] == pid]['frame'].values - selected_sorted[selected_sorted['particle'] == pid]['frame'].values[0]) * 10/60) for pid in particle_ids])
color_mapper = LinearColorMapper(palette=bokeh_palette, low=0, high=max_alive_time)

# Loop through each particle ID and plot its trajectory
for particle_id in particle_ids:
    particle = selected_sorted[selected_sorted['particle'] == particle_id]
    alive_time = (particle['frame'].values - particle['frame'].values[0]) * 10/60
    source = ColumnDataSource(data=dict(x=particle['x'].values * 0.43, y=particle['y'].values * 0.43, color=alive_time))
    p.scatter(x='x', y='y', source=source, size=8, color=transform('color', color_mapper))

# Invert y-axis and set plot limits
p.x_range.start, p.x_range.end = 0, 2048 * 0.43
p.y_range.start, p.y_range.end = 0, 2048 * 0.43

# Add colorbar
color_bar = ColorBar(color_mapper=color_mapper, width=370, location=(0,0), title='Time (minutes)', ticker=FixedTicker(ticks=[0, max_alive_time/2, max_alive_time]))
color_bar.major_label_text_font_size = '16pt'
color_bar.title_text_font_size = '18pt'
p.add_layout(color_bar, 'above')

# Adjust other plot properties
p.y_range.flipped = True
p.xaxis.axis_label = "X (µm)"
p.yaxis.axis_label = "Y (µm)"
p.xaxis.axis_label_text_font_size = '18pt'
p.yaxis.axis_label_text_font_size = '18pt'
p.xaxis.major_label_text_font_size = '16pt'
p.yaxis.major_label_text_font_size = '16pt'
p.xaxis.ticker = FixedTicker(ticks=[0, 440, 880])
p.yaxis.ticker = FixedTicker(ticks=[0, 440, 880])

show(p)


In [ ]:
from bokeh.models import ColorBar, LinearColorMapper, FixedTicker, ColumnDataSource
from bokeh.plotting import figure, output_notebook, show
from bokeh.transform import transform

# Activate inline plotting in Jupyter
output_notebook()

# Extract RGB values from Matplotlib's 'jet' colormap
colormap = plt.get_cmap("jet")
bokeh_palette = [mpl.colors.rgb2hex(m) for m in colormap(np.arange(colormap.N))]

# Calculate max alive time
all_alive_times = []
for particle_id in particle_ids:
    particle = selected_sorted[selected_sorted['particle'] == particle_id]
    alive_time = (particle['frame'].values - particle['frame'].values[0]) * 10/60
    all_alive_times.extend(alive_time)

max_alive_time = np.max(all_alive_times)

# Create a Bokeh figure
p = figure(plot_width=500, plot_height=500, tools="pan,reset,save,wheel_zoom")

# Define color mapper using the 'jet' colormap from Matplotlib
color_mapper = LinearColorMapper(palette=bokeh_palette, low=0, high=max_alive_time)

# Loop through each particle ID and plot
for particle_id in particle_ids:
    particle = selected_sorted[selected_sorted['particle'] == particle_id]
    alive_time = (particle['frame'].values - particle['frame'].values[0]) * 10/60
    source = ColumnDataSource(data=dict(x=particle['x'].values * 0.43, y=particle['y'].values * 0.43, color=alive_time))
    p.scatter(x='x', y='y', source=source, size=8, color=transform('color', color_mapper))

# Adjust plot properties
p.x_range.start, p.x_range.end = 0, 2048 * 0.43
p.y_range.start, p.y_range.end = 0, 2048 * 0.43
p.y_range.flipped = True
p.xaxis.axis_label = "X (µm)"
p.yaxis.axis_label = "Y (µm)"
p.xaxis.axis_label_text_font_size = '18pt'
p.yaxis.axis_label_text_font_size = '18pt'
p.xaxis.major_label_text_font_size = '16pt'
p.yaxis.major_label_text_font_size = '16pt'
p.xaxis.ticker = FixedTicker(ticks=[0, 440, 880])
p.yaxis.ticker = FixedTicker(ticks=[0, 440, 880])
show(p)


In [ ]:
reference_alive_time

In [ ]:
alive_time

In [ ]:
np.max(alive_time)

In [ ]:
max_alive_times

In [ ]:
# List of groups of particles to merge
merge_groups = [[11, 12, 15], [13, 6, 16]]  # Replace with your own groups

# Make a copy of the dataframe to avoid modifying the original one
df_merged = t.copy()

# Iterate over all groups
for group in merge_groups:
    # Get the target ID (the first one in the group)
    target_id = group[0]
    # Get the IDs to be replaced (all others in the group)
    ids_to_replace = group[1:]
    # Replace all IDs in ids_to_replace with the target ID in the 'particle' column
    for particle_id in ids_to_replace:
        df_merged.loc[df_merged['particle'] == particle_id, 'particle'] = target_id


In [ ]:
# A list of sets of particle IDs that you want to merge
particles_to_merge = [{11, 17, 12, 15}, {13, 6, 16}]

merged_particles = pd.DataFrame(columns=t.columns)

for particle_set in particles_to_merge:
    # Concatenate the trajectories of the particles in the current set
    merged_particle = pd.concat(t[t['particle'] == id] for id in particle_set)
    
    # Remove any duplicate entries
    merged_particle = merged_particle.drop_duplicates(subset=['frame'])
    
    # Assign a new particle ID
    merged_particle['particle'] = max(t['particle']) + 1
    
    # Append the merged particle to the DataFrame of merged particles
    merged_particles = merged_particles.append(merged_particle, ignore_index=True)

# Append the DataFrame of merged particles to the original DataFrame
t = t.append(merged_particles, ignore_index=True)

# Remove the original particles
for particle_set in particles_to_merge:
    t = t[~t['particle'].isin(particle_set)]

In [ ]:
def plot_trajectory_with_labels(particles_df):
    """
    This function plots the trajectories of particles and annotates each one with its ID.

    Parameters:
    particles_df: A pandas DataFrame that contains the particles identified by trackpy.
    """
    fig, ax = plt.subplots(figsize=(10, 10))
    tp.plot_traj(particles_df, ax=ax)

    # Get the last position of each particle to place the text
    last_positions = particles_df.groupby('particle').last()

    # Annotate the ID of each particle
    for particle, position in last_positions.iterrows():
        ax.text(position.x, position.y, str(int(particle)), color='red')

    plt.show()

# Then you would use it like this:
plot_trajectory_with_labels(t)


In [ ]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)
print(selected)


In [ ]:
particle_ids = [11, 12, 15, 13, 6, 16]  # Replace with the IDs of the particles you want to plot
selected = t[t['particle'].isin(particle_ids)]
# Filter rows where particle is 24 and index is between 50 and 60
# Print the filtered dataframe
tp.plot_traj(selected, superimpose=frames_thresh[70])

# Ensure data is sorted by frame
selected_sorted = selected.sort_values(by='frame')

# Initialize empty list for storing particle data
particle_data = []

# Loop through each particle ID
for particle_id in particle_ids:
    # Filter for current particle ID
    particle = selected_sorted[selected_sorted['particle'] == particle_id]
    
    # Store x, y, and frame data for current particle
    # If you know the time interval between frames, multiply 'frame' by this interval
    # For example, if each frame is 10 seconds apart, use particle['frame'].values * 10
    particle_data.append((particle['x'].values, particle['y'].values, particle['frame'].values))
    
# Convert list of tuples to numpy array
particle_data = np.array(particle_data)

# Loop through each particle and plot its trajectory
for i in range(len(particle_data)):
    sc = plt.scatter(particle_data[i, 0], particle_data[i, 1], c=particle_data[i, 2], cmap='jet', label=f'Particle {particle_ids[i]}')
plt.colorbar(sc, label='Frame')  # If you converted frames to time, change label to 'Time'

# Invert y-axis and set plot limits
plt.gca().invert_yaxis()
plt.xlim(0, 2048)
plt.ylim(2048, 0)

plt.legend()  # show the legend
plt.show()


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# Assuming your DataFrame is called df

# Filter out frames greater than 100
df_filtered = selected[selected['frame'] <= 70]

plt.figure(figsize=(10, 10))

plt.scatter(df_filtered['x'], df_filtered['y'], c=df_filtered['frame'], cmap='jet')

plt.xlabel('X')
plt.ylabel('Y')
plt.title('XY positions from frame 1 to 100')
plt.xlim(0, 2048) # Set the range of x-axis
plt.ylim(0, 2048) # Set the range of y-axis
plt.colorbar(label='Frame number')
plt.show()


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

particle_ids = [11, 12, 15, 13, 6, 16]  # Replace with the IDs of the particles you want to plot
selected = t[t['particle'].isin(particle_ids)]

# Ensure data is sorted by frame
selected_sorted = selected.sort_values(by='frame')

# Initialize empty list for storing particle data
particle_data = []

# Loop through each particle ID
for particle_id in particle_ids:
    # Filter for current particle ID
    particle = selected_sorted[selected_sorted['particle'] == particle_id]
    
    # Store x, y, and frame data for current particle
    particle_data.append((particle['x'].values, particle['y'].values, particle['frame'].values))
    
# Convert list of tuples to numpy array
particle_data = np.array(particle_data)

# Loop through each particle and plot its trajectory
for i in range(len(particle_data)):
    plt.scatter(particle_data[i, 0], particle_data[i, 1], c=particle_data[i, 2], cmap='jet')

plt.colorbar(label='Frame')

# Invert y-axis and set plot limits
plt.gca().invert_yaxis()
plt.xlim(0, 2048)
plt.ylim(2048, 0)

plt.show()


In [ ]:
print(selected_sorted)

In [ ]:
import matplotlib.ticker as ticker

# Ensure data is sorted by frame
selected_sorted = selected.sort_values(by='frame')

# Initialize empty list for storing particle data
particle_data = []

# Loop through each particle ID
for particle_id in particle_ids:
    # Filter for current particle ID
    particle = selected_sorted[selected_sorted['particle'] == particle_id]
    
    # Store x, y, and time data for current particle. Convert pixels to um.
    particle_data.append((particle['x'].values * 0.43, particle['y'].values * 0.43, particle['frame'].values * 10))
    
# Convert list of tuples to numpy array
particle_data = np.array(particle_data)

# Create figure and axis
fig, ax = plt.subplots(figsize=(6, 5))  # Adjust as needed to get a square plot

# Loop through each particle and plot its trajectory
for i in range(len(particle_data)):
    sc = ax.scatter(particle_data[i, 0], particle_data[i, 1], c=particle_data[i, 2], cmap='jet')

# Invert y-axis and set plot limits
ax.set_xlim(0, 2048 * 0.43)
ax.set_ylim(0, 2048 * 0.43)
ax.tick_params(axis='both', labelsize=20)


# Set x and y ticks
ax.set_xticks([0, 2048 * 0.43 / 2, 2048 * 0.43])
ax.set_yticks([0, 2048 * 0.43 / 2, 2048 * 0.43])

formatter = ticker.FormatStrFormatter('%.0f')
# Apply the formatter to the x and y axes
ax.xaxis.set_major_formatter(formatter)
ax.yaxis.set_major_formatter(formatter)

# Set x and y labels
ax.set_xlabel('X (um)', fontsize=20)
ax.set_ylabel('Y (um)', fontsize=20)

# Add colorbar
cbar = plt.colorbar(sc, ax=ax)
cbar.set_label('Time (s)', fontsize=20)

plt.tight_layout()
plt.show()


# second panel

In [ ]:
# load tiff stake with pims. Notice here I only have two channels, gfp and cy5
#Burst /Volumes/SL_2023/0313/droplet_different_low_MT_6_8_2_burst

frames = pims.ImageSequence('/Users/scliu/Dropbox (Personal)/Academics/PhD_phase/Thomson_Lab/local to global pre-print/data/fig_4/local_move/*.tif')




In [ ]:
bf_frames = frames
bf_frames[1]

In [ ]:
from skimage import filters, util
from scipy import ndimage
import numpy as np

def create_pipeline():
    frame_counter = {'count': 0}  # use a dict to make it mutable inside the nested function

    @pims.pipeline
    def preprocess_foam(img):
        """
        Apply image processing functions to return a binary image
        """
        # Apply thresholds
        percentile = 97 if frame_counter['count'] < 29 else 97  # adjust the percentile based on the frame number
        threshold_value = np.percentile(img, percentile)  # calculate the percentile
        idx = img < threshold_value  # consider pixels brighter than the percentile
        idx2 = img >= threshold_value  # consider pixels not brighter than the percentile
        img[idx] = 255
        img[idx2] = 0
        img = ndimage.binary_dilation(img)
        img = ndimage.binary_dilation(img)

        # increment the frame counter
        frame_counter['count'] += 1

        return util.img_as_int(img)
    
    return preprocess_foam

preprocess_foam = create_pipeline()



frames_thresh = preprocess_foam(bf_frames)
plt.imshow(frames_thresh[120]);

In [ ]:
plt.imshow(frames_thresh[70]);

In [ ]:
img_example = frames_thresh[70]

# Label elements on the picture
white = 255
label_image, number_of_labels = skimage.measure.label(img_example, background=white, return_num=True)
print("Found %d features"%(number_of_labels))
fig, ax = plt.subplots(ncols=1, nrows=1, figsize=(12, 12))
ax.imshow(img_example)
for region in skimage.measure.regionprops(label_image, intensity_image=img_example):
    # Everywhere, skip small and large areas
    if region.area < 2000 or region.area > 50000:
        continue
    # Only black areas
    if region.mean_intensity > 1:
        continue
    # On the top, skip small area with a second threshold
    if region.centroid[0] > 50000 and region.area < 2000:
        continue


    # Draw rectangle which survived to the criterions
    minr, minc, maxr, maxc = region.bbox
    rect = mpatches.Rectangle((minc, minr), maxc - minc, maxr - minr,
                              fill=False, edgecolor='red', linewidth=1)

    ax.add_patch(rect)

In [ ]:

def filter_regions(num, img):
    white = 255
    label_image = skimage.measure.label(img, background=white)
    regions = skimage.measure.regionprops(label_image, intensity_image=img)

    filtered_regions = [
        {
            'y': region.centroid[0],
            'x': region.centroid[1],
            'frame': num
        }
        for region in regions
        if (
            2000 <= region.area <= 50000 and
            region.mean_intensity <= 1 and
            not (region.centroid[0] > 50000 and region.area < 2000)
            )
    ]

    return filtered_regions


filtered_regions_list = [filter_regions(num, img) for num, img in enumerate(frames_thresh)]
features = pd.DataFrame([item for sublist in filtered_regions_list for item in sublist])


In [ ]:
def plot_features_with_labels(frame, features_df, image):
    """
    This function annotates each detected feature with its ID on the plot.

    Parameters:
    frame: The number of the frame you want to plot.
    features_df: A pandas DataFrame that contains the features detected by trackpy.
    image: The image you want to plot the features on.
    """
    # Create a new DataFrame that only contains the features in the frame you're interested in
    frame_features = features_df[features_df.frame==frame]
    
    fig, ax = plt.subplots(figsize=(12, 12))
    ax.imshow(image)
    tp.annotate(frame_features, image, ax=ax)
    
    # Annotate the ID of each feature
    for index, feature in frame_features.iterrows():
        ax.text(feature.x, feature.y, str(index), color='red')

    plt.show()


# Then you would use it like this:
id_example = 1
plot_features_with_labels(18, features, img_example)


In [ ]:
search_range = 100
t = tp.link_df(features, search_range, memory=20)
tp.plot_traj(t, superimpose=frames_thresh[100])